# pystac-client Download Assets

This notebook shows how to use pystac-client to download assets in STAC Items locally, while updating the STAC Item metadata to point to the local files.

In [ ]:
!pip install planetary_computer

In [ ]:
# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.DEBUG)

# Client

We first connect to an API by retrieving the root catalog, or landing page, of the API with the `Client.open` function.

In [ ]:
from pystac import ItemCollection
from pystac_client import Client

# STAC API root URL
URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

# custom headers
headers = []

cat = Client.open(URL, headers=headers)
cat

# API Search

If the Catalog is an API, we have the ability to search for items based on spatio-temporal properties.

In [ ]:
# AOI around Delfzijl, in northern Netherlands
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          6.42425537109375,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.174765470134616
        ]
      ]
    ]
}

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    collections = "sentinel-2-l2a",
    intersects = geom,
    datetime = "2020-01-01/2020-12-31",
    max_items = 5,
    limit = 10
)

# PySTAC ItemCollection
items = search.get_all_items()

# Dictionary (GeoJSON FeatureCollection)
#item_json = items.to_dict()

len(items)

In [ ]:
import planetary_computer as pc

items = pc.sign(items)

In [ ]:
from pystac_client.item_client import ItemClient

# super hacky (need to update item_search.py to return an ItemCollectionClient)
for i in items:
    i.__class__ = ItemClient
items[0]

In [ ]:
import asyncio

async def download_items_assets(items, **kwargs):
    tasks = []
    print(kwargs)
    for item in items:
        task = asyncio.create_task(item.download_assets(**kwargs))
        tasks.append(task)
    new_items = await asyncio.wait(tasks)
    return new_items

In [ ]:
assets = ['B03', 'B04']

#new_items = [download_items_assets(i, assets=assets) for i in items]
new_item = await download_items_assets(items, assets=assets, concurrent_downloads=5) #, overwrite=True)

#await asyncio.wait(new_items)